In [1]:
import re, os
import glob
import datetime

import h5py
import pyart
import pyproj
import netCDF4
import numpy as np
import xarray as xr
import pandas as pd

import dask
import dask.bag as db
from dask.diagnostics import ProgressBar

In [2]:
class NoPrecipitationError(Exception):
    pass


def get_gpm_orbit(gpmfile):
    '''
    Parameters:
    ----------
    gpmfile: str
        GPM data file.

    Returns:
    --------
    orbit: int
        GPM Granule Number.
    '''
    try:
        with h5py.File(gpmfile) as hid:
            grannb = [s for s in hid.attrs['FileHeader'].split() if b'GranuleNumber' in s][0].decode('utf-8')
            orbit = re.findall('[0-9]{3,}', grannb)[0]
    except Exception:
        return 0

    return int(orbit)


def read_GPM(infile, refl_min_thld=17):
    '''
    Read GPM data and organize them into a Dataset.

    Parameters:
    ----------
    gpmfile: str
        GPM data file.
    refl_min_thld: float
        Minimum threshold applied to GPM reflectivity.

    Returns:
    --------
    dset: xr.Dataset
        GPM dataset.
    '''
    data = dict()
    date = dict()
    with h5py.File(infile, 'r') as hid:
        keys = hid['/NS'].keys()
        for k in keys:
            if k == 'Latitude' or k == 'Longitude':
                dims = tuple(hid[f'/NS/{k}'].attrs['DimensionNames'].decode('UTF-8').split(','))
                fv =  hid[f'/NS/{k}'].attrs['_FillValue']
                data[k] = (dims, np.ma.masked_equal(hid[f'/NS/{k}'][:], fv))
            else:
                subkeys = hid[f'/NS/{k}'].keys()
                for sk in subkeys:
                    dims = tuple(hid[f'/NS/{k}/{sk}'].attrs['DimensionNames'].decode('UTF-8').split(','))
                    fv =  hid[f'/NS/{k}/{sk}'].attrs['_FillValue']

                    if sk in ['Year', 'Month', 'DayOfMonth', 'Hour', 'Minute', 'Second', 'MilliSecond']:
                        date[sk] = np.ma.masked_equal(hid[f'/NS/{k}/{sk}'][:], fv)
                    elif sk in ['DayOfYear', 'SecondOfDay']:
                        continue
                    elif sk == 'typePrecip':
                        # Simplify precipitation type
                        data[sk] = (dims, hid[f'/NS/{k}/{sk}'][:] / 10000000)
                    elif sk == 'zFactorCorrected':
                        # Reverse direction along the beam.
                        data[sk] = (dims, np.ma.masked_less_equal(hid[f'/NS/{k}/{sk}'][:][:, :, ::-1], refl_min_thld))
                    elif sk in ['heightBB', 'qualityBB', 'qualityTypePrecip', 'flagPrecip']:
                        data[sk] = (dims, np.ma.masked_equal(hid[f'/NS/{k}/{sk}'][:], fv))
                    else:
                        continue

    try:
        data['zFactorCorrected']
    except Exception:
        raise KeyError(f"GPM Reflectivity not found in {infile}")

    # Create Quality indicator.
    quality = np.zeros(data['heightBB'][-1].shape, dtype=np.int32)
    quality[((data['qualityBB'][-1] == 0) | (data['qualityBB'][-1] == 1)) & (data['qualityTypePrecip'][-1] == 1)] = 1
    quality[(data['qualityBB'][-1] > 1) | (data['qualityTypePrecip'][-1] > 1)] = 2
    data['quality'] = (data['heightBB'][0], quality)

    # Generate dimensions.
    nray = np.linspace(-17.04, 17.04, 49)
    nbin = np.arange(0, 125 * 176, 125)

    R, A = np.meshgrid(nbin, nray)
    distance_from_sr = 407000 / np.cos(np.deg2rad(A)) - R  # called rt in IDL code.
    data['distance_from_sr'] = (('nray', 'nbin'), distance_from_sr)
    
    try:
        _ = date['MilliSecond']
    except KeyError:
        date['MilliSecond'] = date['Second']

    dtime = np.array([datetime.datetime(*d) for d in zip(date['Year'],
                                                         date['Month'],
                                                         date['DayOfMonth'],
                                                         date['Hour'],
                                                         date['Minute'],
                                                         date['Second'],
                                                         date['MilliSecond'])], dtype='datetime64')

    data['nscan'] = (('nscan'), dtime)
    data['nray'] = (('nray'), nray)
    data['nbin'] = (('nbin'), nbin)

    dset = xr.Dataset(data)

    dset.nray.attrs = {'units': 'degree', 'description':'Deviation from Nadir'}
    dset.nbin.attrs = {'units': 'm', 'description':'Downward from 0: TOA to Earth ellipsoid.'}
    dset.attrs['altitude'] = 407000
    dset.attrs['altitude_units'] = 'm'
    dset.attrs['altitude_description'] = "GPM orbit"
    dset.attrs['beamwidth'] = 0.71
    dset.attrs['beamwidth_units'] = 'degree'
    dset.attrs['beamwidth_description'] = "GPM beamwidth"
    dset.attrs['dr'] = 125
    dset.attrs['dr_units'] = 'm'
    dset.attrs['dr_description'] = "GPM gate spacing"
    dset.attrs['orbit'] = get_gpm_orbit(infile)

    return dset


In [3]:
def precip_in_domain(gpmset, grlon, grlat, rmax=150e3, rmin=20e3):
    georef = pyproj.Proj(f"+proj=aeqd +lon_0={grlon} +lat_0={grlat} +ellps=WGS84")
    gpmlat = gpmset.Latitude.values
    gpmlon = gpmset.Longitude.values

    xgpm, ygpm = georef(gpmlon, gpmlat)
    rproj_gpm = (xgpm ** 2 + ygpm ** 2) ** 0.5

    gr_domain = (rproj_gpm <= rmax) & (rproj_gpm >= rmin)
    if gr_domain.sum() < 10:
        info = f'The closest satellite measurement is {np.min(rproj_gpm / 1e3):0.4} km away from ground radar.'
        if gr_domain.sum() == 0:
            raise NoPrecipitationError('GPM swath does not go through the radar domain. ' + info)
        else:
            raise NoPrecipitationError('GPM swath is on the edge of the ground radar domain and there is not enough measurements inside it. ' + info)

    nprof = np.sum(gpmset.flagPrecip.values[gr_domain])
    if nprof < 10:
        raise NoPrecipitationError('No precipitation measured by GPM inside radar domain.')
        
    newset = gpmset.merge({'range_from_gr': (('nscan', 'nray'), rproj_gpm)})
    
    gpmtime0 = newset.nscan.where(newset.range_from_gr == newset.range_from_gr.min()).values.astype('datetime64[s]')
    gpmtime0 = gpmtime0[~np.isnat(gpmtime0)][0]
    
    del newset
    return nprof, gpmtime0

In [4]:
def get_overpass_with_precip(gpmfile, radarset):
    gpmset = read_GPM(gpmfile)
    data = dict()
    for n in range(len(radarset)):
        rid = radarset.id[n]
        rname = radarset.short_name[n]
        grlat = radarset.site_lat[n]
        grlon = radarset.site_lon[n]

        try:
            nprof, gpmtime = precip_in_domain(gpmset, grlat=grlat, grlon=grlon)
        except NoPrecipitationError:
            continue        
        # print(f'{rid} - {rname} radar at ({grlon}, {grlat}) has {nprof} matches with GPM at {gpmtime}.')

        data[rid] = (str(gpmtime),                     
                     rname,
                     str(grlon),
                     str(grlat),
                     str(nprof),
                     gpmfile)
    return data


In [5]:
# flist = np.array([])
# for year in [2018, 2019, 2020]:
#     flist = np.append(flist, sorted(glob.glob(f'/g/data/rq0/admin/calibration/sr_data/gpm_data/{year}/**/**/*.*')))

In [6]:
# flist = sorted(glob.glob('/g/data/rq0/admin/calibration/sr_data/gpm_data/2017/**/**/*.*'))
flist = sorted(glob.glob('/g/data/rq0/admin/calibration/sr_data/trmm_data/2011/*.*'))

In [ ]:
df = pd.read_csv('./radar_site_list.csv')
ndf = df.drop_duplicates('id', keep='last').reset_index()
argslist = [(f, ndf) for f in flist]

In [7]:
bag = db.from_sequence(argslist).starmap(get_overpass_with_precip)
with ProgressBar():
    rslt = bag.compute()

[########################################] | 100% Completed | 36min 32.0s


In [8]:
for n in rslt:
    if len(n) == 0:
        continue
    for rid in n.keys():
        outpath = '/scratch/kl02/vhl548/trmm_output/overpass'
        outfile = os.path.join(outpath, f'trmm.{rid:02}.csv')
        with open(outfile, 'a+') as fid:
            fid.write(','.join(n[rid]))
            fid.write('\n')

In [9]:
2+2

4